In [ ]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import os

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/SCRIPTS/SCRIPTS GENERALES PYTHON')
from Funciones_tablas_validacion import *
from Funciones_woe import *
from jsonProcessing import *

import joblib

In [ ]:
#funcion para calificar una base en base a un modelo determinado
def obtener_score(modelo, base, variables):
    '''Función para calificiar al "base" con el "modelo" en cuestion.
    "variables" hace referencia a las variables que usa el modelo para hacer la calificación.'''
    base_output = base.copy()
    X = base_output[variables] ##base sin smote
    #calcular las proba de las predicciones
    try:
        preds = modelo.predict_proba(X) 
    except:
        preds = modelo.predict(X)
    preds = np.array([ preds[i][0] for i in range(len(preds))])
    base_output['score1'] = np.around(preds*1000, decimals = 0)
    return base_output

# Calificar base de rechazados

In [ ]:
#Base a modelar no importa si es con smote o sin smote de los aceptados
variables_del_modelo = [var.replace('_woe','') for var in pd.read_csv('Score-ActivoIMSS/BASE_MODELAR_IMSS_ACT.csv', encoding='latin-1', nrows=1).columns[6:]]
print(variables_del_modelo)

In [ ]:
#Base de idconsulta con la fecha de rechazados 
info_rechazos = pd.read_csv('Análisis de Rechazos/Rechazos IMSS Activos.csv')
info_rechazos.rename(columns={'IdConsulta':'idconsulta'},inplace = True)
print(info_rechazos.shape)
info_rechazos.head(2)

In [ ]:
#Base muestra de caracteristicas rechazados

caracteristicas = pd.read_csv('Análisis de Rechazos/Rechazados_IMSS_Act_Caracteristicas_con_FechaConsulta_Muestra_50%.csv', usecols=['idconsulta']+variables_del_modelo)
caracteristicas.drop_duplicates(subset='idconsulta',inplace=True)
caracteristicas.shape

In [ ]:
info_rechazos = info_rechazos[info_rechazos['idconsulta'].isin(caracteristicas['idconsulta'].unique())].drop_duplicates(subset='idconsulta')
len(info_rechazos)

In [ ]:
base_rechazos = pd.merge(info_rechazos, caracteristicas, how='left', on='idconsulta')
base_rechazos.shape[0] == info_rechazos.shape[0]

In [ ]:
#tabla de woe de la base de entrenamiento
tabla_woe = pd.read_csv('Análisis de Rechazos/Validación/WOE_SCORE_Act_IMSS.csv')
tabla_woe.head()

In [ ]:
#aplicar los woe a la base de rechazados
base_rechazos_woe = asignar_woes(tabla_woe, base_rechazos)
#base_rechazos_woe.to_csv('Base_Rechazados/Base_cn_woe.csv', index=False)

In [ ]:
#calificar la base
#modelo_aceptados = joblib.load('Score-ActivoIMSS/MLP_base_completa.pkl')#Meter el modelo elegido
modelo_aceptados = load_model('/Score-ActivoIMSS/RNR.h5')
# base_rechazos_calificada = obtener_score(modelo_aceptados, base_rechazos_woe, [var+'_woe' for var in variables_del_modelo])
# base_rechazos_calificada.to_csv('Análisis de Rechazos/Validación/Base_Calificada_Rechazos.csv', index=False) #Base Calificada de Rechazos

# Tabla de validación de la base de aceptados

In [4]:
base_aceptados = pd.read_csv('Score-ActivoIMSS/BASE CALIFICADA_RNR.csv')
tabla_validacion(base_aceptados,  10, string_bgi='BGI', string_score='score1')

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,839-953,739,100.0%,694,93.9%,100.0%,45,6.1%,100.0%,6.1%,0.0%,15.4,19.6%
1,782-838,746,90.1%,672,90.1%,87.7%,74,9.9%,97.5%,8.0%,9.9%,9.1,19.1%
2,686-781,749,80.1%,641,85.6%,75.7%,108,14.4%,93.5%,10.2%,17.8%,5.9,18.2%
3,571-685,747,70.1%,613,82.1%,64.3%,134,17.9%,87.6%,12.1%,23.3%,4.6,16.8%
4,478-570,747,60.0%,570,76.3%,53.4%,177,23.7%,80.3%,14.4%,26.9%,3.2,15.1%
5,405-477,739,50.0%,547,74.0%,43.3%,192,26.0%,70.7%,16.3%,27.4%,2.8,13.0%
6,335-404,744,40.1%,518,69.6%,33.6%,226,30.4%,60.2%,18.3%,26.6%,2.3,10.8%
7,262-334,743,30.1%,502,67.6%,24.4%,241,32.4%,47.9%,20.1%,23.5%,2.1,8.2%
8,191-261,748,20.2%,462,61.8%,15.4%,286,38.2%,34.7%,22.1%,19.3%,1.6,5.4%
9,111-190,758,10.2%,407,53.7%,7.2%,351,46.3%,19.1%,24.6%,11.9%,1.2,1.9%


In [5]:
tv1, tv2 = tablas_de_entrenamiento_y_validacion(base_aceptados, 10, string_bgi='BGI', string_score='score1')
display(tv1)
display(tv2)

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,839-953,519,100.0%,482,92.9%,100.0%,37,7.1%,100.0%,7.1%,0.0%,13.0,19.6%
1,782-838,531,90.0%,480,90.4%,87.7%,51,9.6%,97.1%,8.4%,9.3%,9.4,19.4%
2,686-781,520,79.8%,444,85.4%,75.5%,76,14.6%,93.1%,10.4%,17.5%,5.8,18.0%
3,571-685,530,69.9%,442,83.4%,64.3%,88,16.6%,87.1%,12.0%,22.9%,5.0,17.0%
4,478-570,528,59.7%,406,76.9%,53.0%,122,23.1%,80.2%,14.2%,27.2%,3.3,15.3%
5,405-477,544,49.5%,406,74.6%,42.7%,138,25.4%,70.6%,16.1%,27.9%,2.9,13.6%
6,335-404,515,39.1%,360,69.9%,32.4%,155,30.1%,59.8%,18.1%,27.4%,2.3,10.6%
7,262-334,488,29.2%,333,68.2%,23.2%,155,31.8%,47.6%,19.7%,24.4%,2.1,7.8%
8,191-261,528,19.8%,317,60.0%,14.8%,211,40.0%,35.5%,22.0%,20.7%,1.5,5.5%
9,111-190,505,9.7%,264,52.3%,6.7%,241,47.7%,18.9%,24.5%,12.2%,1.1,1.8%


,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,839-943,220,100.0%,212,96.4%,100.0%,8,3.6%,100.0%,3.6%,0.0%,26.5,19.4%
1,782-838,215,90.2%,192,89.3%,87.5%,23,10.7%,98.6%,7.1%,11.1%,8.3,18.4%
2,686-781,229,80.7%,197,86.0%,76.1%,32,14.0%,94.5%,9.5%,18.3%,6.2,18.6%
3,571-685,217,70.5%,171,78.8%,64.5%,46,21.2%,88.8%,12.4%,24.3%,3.7,16.3%
4,478-569,219,60.9%,164,74.9%,54.4%,55,25.1%,80.5%,14.9%,26.2%,3.0,14.7%
5,405-477,195,51.2%,141,72.3%,44.7%,54,27.7%,70.7%,16.8%,26.0%,2.6,11.4%
6,335-404,229,42.5%,158,69.0%,36.3%,71,31.0%,61.1%,19.0%,24.7%,2.2,11.1%
7,262-334,255,32.3%,169,66.3%,27.0%,86,33.7%,48.4%,21.1%,21.4%,2.0,9.2%
8,191-261,220,21.0%,145,65.9%,17.0%,75,34.1%,33.0%,22.5%,16.0%,1.9,5.1%
9,111-190,253,11.2%,143,56.5%,8.5%,110,43.5%,19.6%,24.9%,11.2%,1.3,2.2%


In [12]:
df_RECALIF = pd.read_csv("Análisis de Rechazos/Validación/Base_Calificada_Rechazos_IMSS_ACTIVOS.csv")
df_RECALIF.head(2)

,idconsulta,Fecha,DOM_009_BUR_Sum_woe,DOM_009_BUR_Mean_woe,DOM_009_BUR_Max_woe,EXP_003_Mean_woe,nombreOtorgante_1_EXP_003_Max_woe,tipoCuenta_3_EXP_003_Sum_woe,tipoCuenta_3_EXP_003_Max_woe,tipoContrato_5_EXP_003_Sum_woe,...,tipoCuenta_3_INT_001_MOR_Max_woe,tipoCuenta_3_INT_002_MOR_Max_woe,tipoCuenta_3_INT_003_MOR_Max_woe,tipoCuenta_3_INT_004_MOR_Max_woe,tipoCuenta_3_INT_006_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Sum_woe,tipoContrato_2_INT_006_MOR_Min_woe,tipoContrato_2_INT_004_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Max_woe,score1
0,482024.0,2019-05-28,-0.285385,-0.456481,-0.16478,-0.085248,-0.238839,-0.269569,0.357042,-0.102302,...,0.557349,0.593411,0.639786,0.670860,0.852907,-0.338258,-0.317913,-0.277624,-0.354463,521.0
1,483284.0,2019-05-30,-0.285385,-0.083106,-0.16478,0.353171,0.491946,0.567439,0.357042,0.377536,...,-0.185696,-0.195536,-0.199456,-0.211604,-0.214945,-0.338258,-0.317913,-0.277624,-0.354463,643.0


In [27]:
df_RECALIF = df_RECALIF[df_RECALIF["score1"].notnull()]


In [28]:
## Hacemos las bandas de los score1 para los Rechazados
df_RECALIF["Banda"] = np.where(df_RECALIF["score1"]<=190,1,np.where((df_RECALIF["score1"]>190)&(df_RECALIF["score1"]<=261),2,
                      np.where((df_RECALIF["score1"]>261)&(df_RECALIF["score1"]<=334),3,
                      np.where((df_RECALIF["score1"]>334)&(df_RECALIF["score1"]<=404),4,
                      np.where((df_RECALIF["score1"]>404)&(df_RECALIF["score1"]<=477),5,
                      np.where((df_RECALIF["score1"]>477)&(df_RECALIF["score1"]<=569),6,
                      np.where((df_RECALIF["score1"]>569)&(df_RECALIF["score1"]<=685),7,
                      np.where((df_RECALIF["score1"]>685)&(df_RECALIF["score1"]<=781),8,
                      np.where((df_RECALIF["score1"]>781)&(df_RECALIF["score1"]<=838),9,
                      np.where((df_RECALIF["score1"]>838),10,1009))))))))))

df_RECALIF["Banda2"] = np.where(df_RECALIF["score1"]<=190,"111-190",np.where((df_RECALIF["score1"]>190)&(df_RECALIF["score1"]<=261),"191-261",
                      np.where((df_RECALIF["score1"]>261)&(df_RECALIF["score1"]<=334),"262-334",
                      np.where((df_RECALIF["score1"]>334)&(df_RECALIF["score1"]<=404),"335-404",
                      np.where((df_RECALIF["score1"]>404)&(df_RECALIF["score1"]<=477),"405-477",
                      np.where((df_RECALIF["score1"]>477)&(df_RECALIF["score1"]<=569),"478-570",
                      np.where((df_RECALIF["score1"]>569)&(df_RECALIF["score1"]<=685),"571-685",
                      np.where((df_RECALIF["score1"]>685)&(df_RECALIF["score1"]<=781),"686-781",
                      np.where((df_RECALIF["score1"]>781)&(df_RECALIF["score1"]<=838),"782-838",
                      np.where((df_RECALIF["score1"]>838),"839-953",1009))))))))))

In [62]:
pd.set_option("display.Max_columns",None)
Dist = pd.DataFrame(df_RECALIF.groupby(["Banda","Banda2"])["Banda"].count())

In [64]:
Dist.rename(columns={"Banda":"Conteo"},inplace=True)
Dist = Dist.rename_axis(None, axis=1).reset_index()
Dist.head(1)

,Banda,Banda2,Conteo
0,1,111-190,563


In [70]:
Dist["%Población"] = (Dist["Conteo"]/Dist["Conteo"].sum()).map(lambda n: '{:,.2%}'.format(n))


In [74]:
Dist = Dist.sort_index(ascending=False)
Dist

,Banda,Banda2,Conteo,%Población
9,10,839-953,125,3.37%
8,9,782-838,176,4.75%
7,8,686-781,242,6.53%
6,7,571-685,315,8.50%
5,6,478-570,382,10.31%
4,5,405-477,435,11.74%
3,4,335-404,467,12.60%
2,3,262-334,480,12.95%
1,2,191-261,521,14.06%
0,1,111-190,563,15.19%


In [75]:
df_RECALIF.head(2)

,idconsulta,Fecha,DOM_009_BUR_Sum_woe,DOM_009_BUR_Mean_woe,DOM_009_BUR_Max_woe,EXP_003_Mean_woe,nombreOtorgante_1_EXP_003_Max_woe,tipoCuenta_3_EXP_003_Sum_woe,tipoCuenta_3_EXP_003_Max_woe,tipoContrato_5_EXP_003_Sum_woe,RES_018_BUR_woe,RES_033_BUR_woe,INT_002_MOR_Mean_woe,INT_003_MOR_Mean_woe,INT_004_MOR_Mean_woe,INT_005_MOR_Mean_woe,INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_004_MOR_Mean_woe,nombreOtorgante_1_INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_003_MOR_Max_woe,nombreOtorgante_1_INT_004_MOR_Max_woe,nombreOtorgante_1_INT_005_MOR_Max_woe,nombreOtorgante_1_INT_006_MOR_Max_woe,tipoCuenta_3_INT_005_MOR_Sum_woe,tipoCuenta_3_INT_006_MOR_Sum_woe,tipoCuenta_1_INT_004_MOR_Mean_woe,tipoCuenta_1_INT_005_MOR_Mean_woe,tipoCuenta_1_INT_006_MOR_Mean_woe,tipoCuenta_3_INT_001_MOR_Max_woe,tipoCuenta_3_INT_002_MOR_Max_woe,tipoCuenta_3_INT_003_MOR_Max_woe,tipoCuenta_3_INT_004_MOR_Max_woe,tipoCuenta_3_INT_006_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Sum_woe,tipoContrato_2_INT_006_MOR_Min_woe,tipoContrato_2_INT_004_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Max_woe,score1,Banda,Banda2
0,482024.0,2019-05-28,-0.285385,-0.456481,-0.16478,-0.085248,-0.238839,-0.269569,0.357042,-0.102302,0.257577,0.334524,-0.103775,-0.123624,-0.09467,-0.088842,-0.09824,-0.228187,-0.264624,-0.289632,-0.294112,-0.26263,-0.240618,0.595365,0.628301,0.005635,0.017526,0.13300,0.557349,0.593411,0.639786,0.670860,0.852907,-0.338258,-0.317913,-0.277624,-0.354463,521.0,6,478-570
1,483284.0,2019-05-30,-0.285385,-0.083106,-0.16478,0.353171,0.491946,0.567439,0.357042,0.377536,0.257577,0.334524,-0.103775,-0.123624,-0.09467,-0.088842,-0.09824,-0.228187,-0.264624,-0.289632,-0.294112,-0.26263,-0.240618,-0.190489,-0.249344,0.005635,0.017526,-0.25212,-0.185696,-0.195536,-0.199456,-0.211604,-0.214945,-0.338258,-0.317913,-0.277624,-0.354463,643.0,7,571-685


In [164]:
from random import seed, shuffle 
#se hace un diccionario donde la llave es la banda identificada y el valor del porcentaje de malos correspondiente a la banda
dic_PM = {1:0.463, 2:0.382, 3:0.324, 4:0.304, 5:0.26, 6:0.237, 7:0.179, 8:0.144, 9:0.099, 10:0.061} #cambiar a acuerdo a la Tv de aceptados
lst_bgi_inferido = []
for banda, tm in list(dic_PM.items()):
    base_banda = df_RECALIF.query(f" Banda == {banda}").reset_index(drop=True)
    num_malos = round(tm * base_banda.shape[0])
    num_buenos = base_banda.shape[0] - num_malos
    bgi_inderido = num_malos * [1] + num_buenos * [0]
    seed(0); shuffle(bgi_inderido)
    lst_bgi_inferido.append(pd.concat([base_banda, pd.Series(bgi_inderido, name='BGI')], axis=1))
base_rechazos_bgi_inferido = pd.concat(lst_bgi_inferido)
base_rechazos_bgi_inferido.shape



(3706, 41)

In [167]:
base_rechazos_bgi_inferido.groupby(["Banda2","Banda"])["BGI"].sum()

Banda2   Banda
111-190  1        261
191-261  2        199
262-334  3        156
335-404  4        142
405-477  5        113
478-570  6         91
571-685  7         56
686-781  8         35
782-838  9         17
839-953  10         8
Name: BGI, dtype: int64

In [113]:
pd.concat()

In [181]:
print(base_aceptados.shape)
print(base_rechazos_bgi_inferido.shape)
display(base_aceptados.head(3))
display(base_rechazos_bgi_inferido.head(3))

(7460, 43)
(3706, 42)


,idconsulta,idcredit,BGI,IND_2,DOM_009_BUR_Sum_woe,DOM_009_BUR_Mean_woe,DOM_009_BUR_Max_woe,EXP_003_Mean_woe,nombreOtorgante_1_EXP_003_Max_woe,tipoCuenta_3_EXP_003_Sum_woe,tipoCuenta_3_EXP_003_Max_woe,tipoContrato_5_EXP_003_Sum_woe,RES_018_BUR_woe,RES_033_BUR_woe,INT_002_MOR_Mean_woe,INT_003_MOR_Mean_woe,INT_004_MOR_Mean_woe,INT_005_MOR_Mean_woe,INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_004_MOR_Mean_woe,nombreOtorgante_1_INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_003_MOR_Max_woe,nombreOtorgante_1_INT_004_MOR_Max_woe,nombreOtorgante_1_INT_005_MOR_Max_woe,nombreOtorgante_1_INT_006_MOR_Max_woe,tipoCuenta_3_INT_005_MOR_Sum_woe,tipoCuenta_3_INT_006_MOR_Sum_woe,tipoCuenta_1_INT_004_MOR_Mean_woe,tipoCuenta_1_INT_005_MOR_Mean_woe,tipoCuenta_1_INT_006_MOR_Mean_woe,tipoCuenta_3_INT_001_MOR_Max_woe,tipoCuenta_3_INT_002_MOR_Max_woe,tipoCuenta_3_INT_003_MOR_Max_woe,tipoCuenta_3_INT_004_MOR_Max_woe,tipoCuenta_3_INT_006_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Sum_woe,tipoContrato_2_INT_006_MOR_Min_woe,tipoContrato_2_INT_004_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Max_woe,Partición,score1,BGI_CONS,score1_TILN
0,895247,660294,0,2021_1,-0.285385,-0.083106,-0.16478,0.353171,0.491946,0.090153,0.357042,0.377536,-0.692360,-0.095220,0.328488,0.434233,0.426897,0.390760,0.421439,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,0.595365,0.628301,0.417649,0.464799,0.521675,0.557349,0.593411,0.639786,0.670860,0.852907,-0.064847,-0.064847,-0.064847,-0.064847,1_Entrenamiento,833.0,0,9
1,895254,664489,0,2021_1,0.118538,0.301960,0.30902,0.353171,0.491946,0.090153,0.357042,-0.102302,0.257577,0.334524,0.328488,0.434233,0.426897,0.390760,0.421439,0.278216,-0.264624,0.426413,-0.085305,-0.014613,-0.240618,0.096370,-0.032230,0.107306,0.107306,0.107306,0.557349,0.593411,0.639786,0.118394,0.206295,-0.058778,0.197724,-0.011310,0.043615,2_Comprobación,754.0,0,8
2,895289,664187,0,2021_1,-0.285385,-0.456481,-0.16478,-0.367446,-0.238839,-0.269569,-0.141695,-0.433983,0.257577,-0.095220,-0.344724,-0.123624,-0.348643,-0.385081,-0.402661,-0.228187,-0.264624,-0.119415,-0.085305,-0.262630,-0.240618,-0.190489,-0.249344,-0.328481,-0.361794,-0.252120,-0.185696,-0.195536,-0.199456,-0.211604,-0.214945,-0.064847,-0.064847,-0.064847,-0.064847,1_Entrenamiento,344.0,0,4


,idconsulta,Fecha,DOM_009_BUR_Sum_woe,DOM_009_BUR_Mean_woe,DOM_009_BUR_Max_woe,EXP_003_Mean_woe,nombreOtorgante_1_EXP_003_Max_woe,tipoCuenta_3_EXP_003_Sum_woe,tipoCuenta_3_EXP_003_Max_woe,tipoContrato_5_EXP_003_Sum_woe,RES_018_BUR_woe,RES_033_BUR_woe,INT_002_MOR_Mean_woe,INT_003_MOR_Mean_woe,INT_004_MOR_Mean_woe,INT_005_MOR_Mean_woe,INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_004_MOR_Mean_woe,nombreOtorgante_1_INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_003_MOR_Max_woe,nombreOtorgante_1_INT_004_MOR_Max_woe,nombreOtorgante_1_INT_005_MOR_Max_woe,nombreOtorgante_1_INT_006_MOR_Max_woe,tipoCuenta_3_INT_005_MOR_Sum_woe,tipoCuenta_3_INT_006_MOR_Sum_woe,tipoCuenta_1_INT_004_MOR_Mean_woe,tipoCuenta_1_INT_005_MOR_Mean_woe,tipoCuenta_1_INT_006_MOR_Mean_woe,tipoCuenta_3_INT_001_MOR_Max_woe,tipoCuenta_3_INT_002_MOR_Max_woe,tipoCuenta_3_INT_003_MOR_Max_woe,tipoCuenta_3_INT_004_MOR_Max_woe,tipoCuenta_3_INT_006_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Sum_woe,tipoContrato_2_INT_006_MOR_Min_woe,tipoContrato_2_INT_004_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Max_woe,score1,Banda,Banda2,BGI,IND_2
0,488820.0,2019-06-07,-0.285385,-0.456481,-0.164780,-0.367446,0.086753,-0.216929,-0.216929,-0.102302,-0.191508,-0.095220,-0.344724,-0.486147,-0.348643,-0.385081,-0.402661,-0.228187,-0.264624,-0.289632,-0.294112,-0.262630,-0.240618,-0.134228,-0.134228,-0.328481,-0.361794,-0.25212,-0.134228,-0.134228,-0.134228,-0.134228,-0.134228,-0.338258,-0.317913,-0.277624,-0.354463,168.0,1,111-190,0,2019_6
1,490405.0,2019-06-10,-0.285385,-0.456481,-0.559121,-0.367446,-0.238839,-0.269569,-0.141695,-0.102302,0.257577,-0.748324,-0.103775,-0.123624,-0.094670,-0.088842,-0.098240,-0.228187,-0.264624,-0.289632,-0.294112,-0.262630,-0.240618,-0.190489,-0.032230,0.005635,0.017526,0.13300,-0.185696,0.058448,0.133063,0.118394,-0.214945,-0.338258,-0.317913,-0.277624,-0.354463,128.0,1,111-190,0,2019_6
2,515644.0,2019-07-10,0.457035,0.301960,0.309020,-0.367446,-0.238839,-0.269569,-0.141695,-0.102302,-0.692360,-0.748324,-0.103775,-0.123624,-0.094670,-0.088842,-0.098240,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.134228,-0.134228,0.417649,0.464799,0.13300,-0.134228,-0.134228,-0.134228,-0.134228,-0.134228,-0.064847,-0.064847,-0.064847,-0.064847,163.0,1,111-190,0,2019_7


In [179]:
base_rechazos_bgi_inferido["IND_2"]= (pd.DatetimeIndex(base_rechazos_bgi_inferido["Fecha"]).year).astype(str)+"_"+(pd.DatetimeIndex(base_rechazos_bgi_inferido["Fecha"]).month).astype(str)

In [191]:
#base_rechazos_bgi_inferido.groupby("IND_2")["idconsulta"].count()
base_rechazos_bgi_inferido_sin_3Meses = base_rechazos_bgi_inferido[-base_rechazos_bgi_inferido["IND_2"].isin(["2021_6","2021_7","2021_8"])]

In [193]:
base_aceptados_sin_3Meses = base_aceptados[-base_aceptados["IND_2"].isin(["2021_6","2021_7","2021_8"])]

In [196]:
#Base aceptados 3 Meses para hacer la Validación
base_aceptados_3Meses_v = base_aceptados[base_aceptados["IND_2"].isin(["2021_6","2021_7","2021_8"])]

In [194]:
base_aceptados_sin_3Meses.head()

,idconsulta,idcredit,BGI,IND_2,DOM_009_BUR_Sum_woe,DOM_009_BUR_Mean_woe,DOM_009_BUR_Max_woe,EXP_003_Mean_woe,nombreOtorgante_1_EXP_003_Max_woe,tipoCuenta_3_EXP_003_Sum_woe,tipoCuenta_3_EXP_003_Max_woe,tipoContrato_5_EXP_003_Sum_woe,RES_018_BUR_woe,RES_033_BUR_woe,INT_002_MOR_Mean_woe,INT_003_MOR_Mean_woe,INT_004_MOR_Mean_woe,INT_005_MOR_Mean_woe,INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_004_MOR_Mean_woe,nombreOtorgante_1_INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_003_MOR_Max_woe,nombreOtorgante_1_INT_004_MOR_Max_woe,nombreOtorgante_1_INT_005_MOR_Max_woe,nombreOtorgante_1_INT_006_MOR_Max_woe,tipoCuenta_3_INT_005_MOR_Sum_woe,tipoCuenta_3_INT_006_MOR_Sum_woe,tipoCuenta_1_INT_004_MOR_Mean_woe,tipoCuenta_1_INT_005_MOR_Mean_woe,tipoCuenta_1_INT_006_MOR_Mean_woe,tipoCuenta_3_INT_001_MOR_Max_woe,tipoCuenta_3_INT_002_MOR_Max_woe,tipoCuenta_3_INT_003_MOR_Max_woe,tipoCuenta_3_INT_004_MOR_Max_woe,tipoCuenta_3_INT_006_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Sum_woe,tipoContrato_2_INT_006_MOR_Min_woe,tipoContrato_2_INT_004_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Max_woe,Partición,score1,BGI_CONS,score1_TILN
0,895247,660294,0,2021_1,-0.285385,-0.083106,-0.16478,0.353171,0.491946,0.090153,0.357042,0.377536,-0.692360,-0.095220,0.328488,0.434233,0.426897,0.390760,0.421439,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,0.595365,0.628301,0.417649,0.464799,0.521675,0.557349,0.593411,0.639786,0.670860,0.852907,-0.064847,-0.064847,-0.064847,-0.064847,1_Entrenamiento,833.0,0,9
1,895254,664489,0,2021_1,0.118538,0.301960,0.30902,0.353171,0.491946,0.090153,0.357042,-0.102302,0.257577,0.334524,0.328488,0.434233,0.426897,0.390760,0.421439,0.278216,-0.264624,0.426413,-0.085305,-0.014613,-0.240618,0.096370,-0.032230,0.107306,0.107306,0.107306,0.557349,0.593411,0.639786,0.118394,0.206295,-0.058778,0.197724,-0.011310,0.043615,2_Comprobación,754.0,0,8
2,895289,664187,0,2021_1,-0.285385,-0.456481,-0.16478,-0.367446,-0.238839,-0.269569,-0.141695,-0.433983,0.257577,-0.095220,-0.344724,-0.123624,-0.348643,-0.385081,-0.402661,-0.228187,-0.264624,-0.119415,-0.085305,-0.262630,-0.240618,-0.190489,-0.249344,-0.328481,-0.361794,-0.252120,-0.185696,-0.195536,-0.199456,-0.211604,-0.214945,-0.064847,-0.064847,-0.064847,-0.064847,1_Entrenamiento,344.0,0,4
3,895296,660987,0,2021_1,0.457035,0.301960,0.30902,0.353171,0.491946,-0.216929,-0.216929,-0.102302,0.257577,0.334524,-0.344724,-0.123624,-0.094670,-0.088842,-0.098240,-0.228187,-0.264624,-0.289632,-0.294112,-0.262630,-0.240618,-0.134228,-0.134228,-0.328481,-0.361794,-0.252120,-0.134228,-0.134228,-0.134228,-0.134228,-0.134228,-0.338258,-0.317913,-0.277624,-0.354463,1_Entrenamiento,418.0,0,5
4,895302,661273,1,2021_1,0.118538,0.301960,0.30902,-0.085248,-0.165863,-0.269569,-0.141695,-0.102302,0.257577,0.334524,-0.344724,-0.486147,-0.348643,-0.385081,-0.402661,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.190489,-0.249344,-0.328481,-0.361794,-0.252120,-0.185696,-0.195536,-0.199456,-0.211604,-0.214945,-0.064847,-0.064847,-0.064847,-0.064847,1_Entrenamiento,445.0,1,5


In [197]:
df_rechazados_aceptados_sin_3Meses = pd.concat([base_aceptados_sin_3Meses,base_rechazos_bgi_inferido_sin_3Meses],0)

In [199]:
print(df_rechazados_aceptados_sin_3Meses.shape)
display(df_rechazados_aceptados_sin_3Meses.head())

(9546, 46)


,idconsulta,idcredit,BGI,IND_2,DOM_009_BUR_Sum_woe,DOM_009_BUR_Mean_woe,DOM_009_BUR_Max_woe,EXP_003_Mean_woe,nombreOtorgante_1_EXP_003_Max_woe,tipoCuenta_3_EXP_003_Sum_woe,tipoCuenta_3_EXP_003_Max_woe,tipoContrato_5_EXP_003_Sum_woe,RES_018_BUR_woe,RES_033_BUR_woe,INT_002_MOR_Mean_woe,INT_003_MOR_Mean_woe,INT_004_MOR_Mean_woe,INT_005_MOR_Mean_woe,INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_004_MOR_Mean_woe,nombreOtorgante_1_INT_006_MOR_Mean_woe,nombreOtorgante_1_INT_003_MOR_Max_woe,nombreOtorgante_1_INT_004_MOR_Max_woe,nombreOtorgante_1_INT_005_MOR_Max_woe,nombreOtorgante_1_INT_006_MOR_Max_woe,tipoCuenta_3_INT_005_MOR_Sum_woe,tipoCuenta_3_INT_006_MOR_Sum_woe,tipoCuenta_1_INT_004_MOR_Mean_woe,tipoCuenta_1_INT_005_MOR_Mean_woe,tipoCuenta_1_INT_006_MOR_Mean_woe,tipoCuenta_3_INT_001_MOR_Max_woe,tipoCuenta_3_INT_002_MOR_Max_woe,tipoCuenta_3_INT_003_MOR_Max_woe,tipoCuenta_3_INT_004_MOR_Max_woe,tipoCuenta_3_INT_006_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Sum_woe,tipoContrato_2_INT_006_MOR_Min_woe,tipoContrato_2_INT_004_MOR_Max_woe,tipoContrato_2_INT_006_MOR_Max_woe,Partición,score1,BGI_CONS,score1_TILN,Fecha,Banda,Banda2
0,895247.0,660294.0,0,2021_1,-0.285385,-0.083106,-0.16478,0.353171,0.491946,0.090153,0.357042,0.377536,-0.692360,-0.095220,0.328488,0.434233,0.426897,0.390760,0.421439,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,0.595365,0.628301,0.417649,0.464799,0.521675,0.557349,0.593411,0.639786,0.670860,0.852907,-0.064847,-0.064847,-0.064847,-0.064847,1_Entrenamiento,833.0,0.0,9,NaN,NaN,NaN
1,895254.0,664489.0,0,2021_1,0.118538,0.301960,0.30902,0.353171,0.491946,0.090153,0.357042,-0.102302,0.257577,0.334524,0.328488,0.434233,0.426897,0.390760,0.421439,0.278216,-0.264624,0.426413,-0.085305,-0.014613,-0.240618,0.096370,-0.032230,0.107306,0.107306,0.107306,0.557349,0.593411,0.639786,0.118394,0.206295,-0.058778,0.197724,-0.011310,0.043615,2_Comprobación,754.0,0.0,8,NaN,NaN,NaN
2,895289.0,664187.0,0,2021_1,-0.285385,-0.456481,-0.16478,-0.367446,-0.238839,-0.269569,-0.141695,-0.433983,0.257577,-0.095220,-0.344724,-0.123624,-0.348643,-0.385081,-0.402661,-0.228187,-0.264624,-0.119415,-0.085305,-0.262630,-0.240618,-0.190489,-0.249344,-0.328481,-0.361794,-0.252120,-0.185696,-0.195536,-0.199456,-0.211604,-0.214945,-0.064847,-0.064847,-0.064847,-0.064847,1_Entrenamiento,344.0,0.0,4,NaN,NaN,NaN
3,895296.0,660987.0,0,2021_1,0.457035,0.301960,0.30902,0.353171,0.491946,-0.216929,-0.216929,-0.102302,0.257577,0.334524,-0.344724,-0.123624,-0.094670,-0.088842,-0.098240,-0.228187,-0.264624,-0.289632,-0.294112,-0.262630,-0.240618,-0.134228,-0.134228,-0.328481,-0.361794,-0.252120,-0.134228,-0.134228,-0.134228,-0.134228,-0.134228,-0.338258,-0.317913,-0.277624,-0.354463,1_Entrenamiento,418.0,0.0,5,NaN,NaN,NaN
4,895302.0,661273.0,1,2021_1,0.118538,0.301960,0.30902,-0.085248,-0.165863,-0.269569,-0.141695,-0.102302,0.257577,0.334524,-0.344724,-0.486147,-0.348643,-0.385081,-0.402661,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.097057,-0.190489,-0.249344,-0.328481,-0.361794,-0.252120,-0.185696,-0.195536,-0.199456,-0.211604,-0.214945,-0.064847,-0.064847,-0.064847,-0.064847,1_Entrenamiento,445.0,1.0,5,NaN,NaN,NaN


In [203]:
#df_rechazados_aceptados_sin_3Meses.head(2)
df_rechazados_aceptados_sin_3Meses.drop(["idcredit","BGI_CONS","score1_TILN","Fecha","Banda","Banda2","Partición"],1).to_csv("ActivoIMSS_rechazados_aceptados_sin_3Meses.csv")

In [204]:
print(df_rechazados_aceptados_sin_3Meses["BGI"].sum())
print(df_rechazados_aceptados_sin_3Meses["BGI"].count())

2517
9546


In [142]:
# index_1 = df_RECALIF[df_RECALIF["Banda"]==1].sample(261,random_state=0).index
# df_RECALIF.loc[index_1][["BGI"]]=1
# df_RECALIF[df_RECALIF["Banda"]==1].sample(261,random_state=0)["BGI"] = 1


In [180]:
#df_RECALIF["Banda"]==1




In [141]:
#df_RECALIF.loc[index_1]["BGI"]=1


0

## Politica Felxible

In [ ]:
df_idconsultas = pd.read_csv('Base_Aceptados/Base_Nuevo_Score_JyP_IMSS_v2.csv', usecols=[0,2,5])
id_consultas_pol_flex = df_idconsultas[df_idconsultas['IND_2'].apply(lambda x: tuple(map(int, x.split('_'))) <=(2020,6))]['idconsulta'].unique()

In [ ]:
type(id_consultas_pol_flex[0])

In [ ]:
df_BCScore = pd.DataFrame()
for index in [1,2,3,4]:
    dir_json = f"C:/Users/JoséEmmanuelParedesR/Downloads/Json_NuevoScore_JyP_IMSS/Json{index}/"
    id_lst =[]
    bcscore_lst = []
    print(index,len(os.listdir(dir_json)))
    for i, filename in enumerate(os.listdir(dir_json)):
        print(i, end='\r')
        try:
            cuentas = jsonProcessing(dir_json+ filename).tab('cuentas')[['nombreOtorgante','tipoContrato','formaPagoActual']]
            cuentas = cuentas[(cuentas['nombreOtorgante']=='FINANCIERA') & (cuentas['tipoContrato'] == 'PL') & (cuentas['formaPagoActual']=='01')]
        except: continue
        if cuentas.shape[0] > 0:
            id_lst.append(int(filename.replace('.json','')))
            try: bcs = jsonProcessing(dir_json+filename).tab('scoreBuroCredito')['valorScore'].values[0]
            except: bcs = np.nan
            bcscore_lst.append(bcs)
    df_BCScore = pd.concat([df_BCScore, pd.DataFrame({'idconsulta':id_lst,'BC Score':bcscore_lst})])

In [ ]:
politica_flexible = df_BCScore[(df_BCScore['idconsulta'].isin(id_consultas_pol_flex)) & (df_BCScore['BC Score']>=500)]['idconsulta'].unique()

In [ ]:

tabla_validacion(base_aceptados[base_aceptados['idconsulta'].isin(politica_flexible)], 10, string_bgi='BGI', string_score='score1')

# Notebook

In [ ]:
aceptados = pd.read_excel('Ejercicio Inferencia de Rechazo.xlsx', sheet_name='ACEPTADOS')
aceptados

In [ ]:
aceptados['Fecha Consulta'] = pd.to_datetime(aceptados["Fecha Consulta"], format = '%Y/%m/%d')

In [ ]:
aceptados[aceptados['Fecha Consulta'] < '2020/01/01']

In [ ]:
data =aceptados.drop(columns=['idconsulta', 'Fecha Consulta'])
data

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import metrics 

In [ ]:
from sklearn.model_selection import train_test_split

# División de los datos en train y test
# ==============================================================================
X = data.drop(columns = 'BGI')
y = data['BGI']

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.8,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

In [ ]:
# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo
#X_train = sm.add_constant(X_train, prepend=True)
modelo = sm.Logit(endog=y_train, exog=X_train,)
modelo = modelo.fit()

In [ ]:
# Predicciones con intervalo de confianza 
# ==============================================================================
score = modelo.predict(exog = X)*1000
score = score.astype(int)
score

In [ ]:
aceptados['score']=score
aceptados

In [ ]:
aceptados.score.max()

In [ ]:
aceptados.score.min()

### Punto de corte

In [ ]:
aceptados.BGI.value_counts()

In [ ]:
aceptados[(aceptados.score>659)].BGI.value_counts()#.values[1]

In [ ]:
ceros, unos, x, y = [],[], [], []

for i in range(0,970):
    try:
        uno = aceptados[(aceptados.score>i)].BGI.value_counts()[1]
    except:
        uno = 0
    try:
        cero = aceptados[(aceptados.score>i)].BGI.value_counts()[0]
    except:
        cero = 0
    #print(uno,cero)
    div = (uno) + (cero)
    ceros.append(cero*100/div)
    unos.append(uno*100/div)
    y.append(div)
    x.append(i)




In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x,unos)
plt.show()

### parceling

In [ ]:
rechazados = pd.read_excel('Ejercicio Inferencia de Rechazo.xlsx', sheet_name='RECHAZADOS')
rechazados

In [ ]:
data =rechazados.drop(columns=['idconsulta', 'Fecha Consulta'])
data

In [ ]:
score2 = modelo.predict(exog = data)*1000
score2 = score2.astype(int)
score2

In [ ]:
rechazados['score'] = score2

In [ ]:
aceptados[(aceptados.score>100) & (aceptados.score<150)].BGI.value_counts()[0]

In [ ]:
ceros, unos, x, y = [],[], [], []

for i in range(100,1000,50):
    try:
        uno = aceptados[(aceptados.score>i) & (aceptados.score<i+50)].BGI.value_counts()[1]
    except:
        uno = 0
    try:
        cero = aceptados[(aceptados.score>i) & (aceptados.score<i+50)].BGI.value_counts()[0]
    except:
        cero = 0
    div = (uno) + (cero)
    ceros.append(cero*100/div)
    unos.append(uno*100/div)
    y.append(div)
    x.append(str(i) + '-' + str(i+50))

In [ ]:
rechazados.score.max()

In [ ]:
tot, x2, y2 = [],[], [], []

for i in range(100,1000,50):
    tot.append(len(rechazados[(rechazados.score>i) & (rechazados.score<i+50)]))


In [ ]:
rechm, rechb = [], []
for i in range(len(tot)):
    rechm.append(int(tot[i]*ceros[i]/100))
    rechb.append(int(tot[i]*unos[i]/100))
    if (rechm[i] + rechb[i] == tot[i] ):
        continue
    else:
        rechb[i] += 1

In [ ]:
df = pd.DataFrame()
df['Rango'] = x
df['% Malos'] = ceros
df['% Buenos'] = unos
df['Rechazados'] = unos2
df['Rechazados Malos'] = rechm
df['Rechazados Buenos'] = rechb

In [ ]:
df